# Amazon SageMaker Edge Manager Demo Notebook

**SageMaker Studio Kernel**: Python 3 (PyTorch 1.6 Python 3.6 CPU Optimized)

In this demo, we will compile a pre-trained YoloV4 model using **Amazon SageMaker Neo** and package the model for deployment via **Amazon SageMaker Edge Manager**. This notebook has the following steps:
 - Generate the pre-trained model and upload it to **Amazon S3**
 - Generate IoT credentials required by the Agent
 - Generate the config file required by the Agent
 - Create and upload the full agent deployment package to S3

### Install required libraries and declare dependencies

In [ ]:
!pip install --upgrade pillow pycocotools
!apt-get -y update && apt-get -y install build-essential procps
!pip install -U numpy sysv_ipc grpcio-tools grpcio protobuf

In [ ]:
import sys

!{sys.executable} -m pip install -q --upgrade sagemaker

In [ ]:
import numpy as np
import time
import json
import requests
import boto3
import os
import sagemaker

### Generate pre-trained YoloV4 model

We will be using the following PyTorch implementation of YoloV4 for Object Detection and Classification: https://github.com/Tianxiaomo/pytorch-YOLOv4.

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


class Mish(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x * (torch.tanh(torch.nn.functional.softplus(x)))
        return x


class Upsample(nn.Module):
    def __init__(self):
        super(Upsample, self).__init__()

    def forward(self, x, target_size, inference=False):
        assert (x.data.dim() == 4)

        if inference:

            return x.view(x.size(0), x.size(1), x.size(2), 1, x.size(3), 1).\
                    expand(x.size(0), x.size(1), x.size(2), target_size[2] // x.size(2), x.size(3), target_size[3] // x.size(3)).\
                    contiguous().view(x.size(0), x.size(1), target_size[2], target_size[3])
        else:
            return F.interpolate(x, size=(target_size[2], target_size[3]), mode='nearest')


class Conv_Bn_Activation(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, activation, bn=True, bias=False):
        super().__init__()
        pad = (kernel_size - 1) // 2

        self.conv = nn.ModuleList()
        if bias:
            self.conv.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, pad))
        else:
            self.conv.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, pad, bias=False))
        if bn:
            self.conv.append(nn.BatchNorm2d(out_channels))
        if activation == "mish":
            self.conv.append(Mish())
        elif activation == "relu":
            self.conv.append(nn.ReLU(inplace=True))
        elif activation == "leaky":
            self.conv.append(nn.LeakyReLU(0.1, inplace=True))
        elif activation == "linear":
            pass
        else:
            print("activate error !!! {} {} {}".format(sys._getframe().f_code.co_filename,
                                                       sys._getframe().f_code.co_name, sys._getframe().f_lineno))

    def forward(self, x):
        for l in self.conv:
            x = l(x)
        return x


class ResBlock(nn.Module):
    """
    Sequential residual blocks each of which consists of \
    two convolution layers.
    Args:
        ch (int): number of input and output channels.
        nblocks (int): number of residual blocks.
        shortcut (bool): if True, residual tensor addition is enabled.
    """

    def __init__(self, ch, nblocks=1, shortcut=True):
        super().__init__()
        self.shortcut = shortcut
        self.module_list = nn.ModuleList()
        for i in range(nblocks):
            resblock_one = nn.ModuleList()
            resblock_one.append(Conv_Bn_Activation(ch, ch, 1, 1, 'mish'))
            resblock_one.append(Conv_Bn_Activation(ch, ch, 3, 1, 'mish'))
            self.module_list.append(resblock_one)

    def forward(self, x):
        for module in self.module_list:
            h = x
            for res in module:
                h = res(h)
            x = x + h if self.shortcut else h
        return x


class DownSample1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv_Bn_Activation(3, 32, 3, 1, 'mish')

        self.conv2 = Conv_Bn_Activation(32, 64, 3, 2, 'mish')
        self.conv3 = Conv_Bn_Activation(64, 64, 1, 1, 'mish')
        # [route]
        # layers = -2
        self.conv4 = Conv_Bn_Activation(64, 64, 1, 1, 'mish')

        self.conv5 = Conv_Bn_Activation(64, 32, 1, 1, 'mish')
        self.conv6 = Conv_Bn_Activation(32, 64, 3, 1, 'mish')
        # [shortcut]
        # from=-3
        # activation = linear

        self.conv7 = Conv_Bn_Activation(64, 64, 1, 1, 'mish')
        # [route]
        # layers = -1, -7
        self.conv8 = Conv_Bn_Activation(128, 64, 1, 1, 'mish')

    def forward(self, input):
        x1 = self.conv1(input)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        # route -2
        x4 = self.conv4(x2)
        x5 = self.conv5(x4)
        x6 = self.conv6(x5)
        # shortcut -3
        x6 = x6 + x4

        x7 = self.conv7(x6)
        # [route]
        # layers = -1, -7
        x7 = torch.cat([x7, x3], dim=1)
        x8 = self.conv8(x7)
        return x8


class DownSample2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv_Bn_Activation(64, 128, 3, 2, 'mish')
        self.conv2 = Conv_Bn_Activation(128, 64, 1, 1, 'mish')
        # r -2
        self.conv3 = Conv_Bn_Activation(128, 64, 1, 1, 'mish')

        self.resblock = ResBlock(ch=64, nblocks=2)

        # s -3
        self.conv4 = Conv_Bn_Activation(64, 64, 1, 1, 'mish')
        # r -1 -10
        self.conv5 = Conv_Bn_Activation(128, 128, 1, 1, 'mish')

    def forward(self, input):
        x1 = self.conv1(input)
        x2 = self.conv2(x1)
        x3 = self.conv3(x1)

        r = self.resblock(x3)
        x4 = self.conv4(r)

        x4 = torch.cat([x4, x2], dim=1)
        x5 = self.conv5(x4)
        return x5


class DownSample3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv_Bn_Activation(128, 256, 3, 2, 'mish')
        self.conv2 = Conv_Bn_Activation(256, 128, 1, 1, 'mish')
        self.conv3 = Conv_Bn_Activation(256, 128, 1, 1, 'mish')

        self.resblock = ResBlock(ch=128, nblocks=8)
        self.conv4 = Conv_Bn_Activation(128, 128, 1, 1, 'mish')
        self.conv5 = Conv_Bn_Activation(256, 256, 1, 1, 'mish')

    def forward(self, input):
        x1 = self.conv1(input)
        x2 = self.conv2(x1)
        x3 = self.conv3(x1)

        r = self.resblock(x3)
        x4 = self.conv4(r)

        x4 = torch.cat([x4, x2], dim=1)
        x5 = self.conv5(x4)
        return x5


class DownSample4(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv_Bn_Activation(256, 512, 3, 2, 'mish')
        self.conv2 = Conv_Bn_Activation(512, 256, 1, 1, 'mish')
        self.conv3 = Conv_Bn_Activation(512, 256, 1, 1, 'mish')

        self.resblock = ResBlock(ch=256, nblocks=8)
        self.conv4 = Conv_Bn_Activation(256, 256, 1, 1, 'mish')
        self.conv5 = Conv_Bn_Activation(512, 512, 1, 1, 'mish')

    def forward(self, input):
        x1 = self.conv1(input)
        x2 = self.conv2(x1)
        x3 = self.conv3(x1)

        r = self.resblock(x3)
        x4 = self.conv4(r)

        x4 = torch.cat([x4, x2], dim=1)
        x5 = self.conv5(x4)
        return x5


class DownSample5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv_Bn_Activation(512, 1024, 3, 2, 'mish')
        self.conv2 = Conv_Bn_Activation(1024, 512, 1, 1, 'mish')
        self.conv3 = Conv_Bn_Activation(1024, 512, 1, 1, 'mish')

        self.resblock = ResBlock(ch=512, nblocks=4)
        self.conv4 = Conv_Bn_Activation(512, 512, 1, 1, 'mish')
        self.conv5 = Conv_Bn_Activation(1024, 1024, 1, 1, 'mish')

    def forward(self, input):
        x1 = self.conv1(input)
        x2 = self.conv2(x1)
        x3 = self.conv3(x1)

        r = self.resblock(x3)
        x4 = self.conv4(r)

        x4 = torch.cat([x4, x2], dim=1)
        x5 = self.conv5(x4)
        return x5


class Neck(nn.Module):
    def __init__(self, inference=False):
        super().__init__()
        self.inference = inference

        self.conv1 = Conv_Bn_Activation(1024, 512, 1, 1, 'leaky')
        self.conv2 = Conv_Bn_Activation(512, 1024, 3, 1, 'leaky')
        self.conv3 = Conv_Bn_Activation(1024, 512, 1, 1, 'leaky')
        # SPP
        self.maxpool1 = nn.MaxPool2d(kernel_size=5, stride=1, padding=5 // 2)
        self.maxpool2 = nn.MaxPool2d(kernel_size=9, stride=1, padding=9 // 2)
        self.maxpool3 = nn.MaxPool2d(kernel_size=13, stride=1, padding=13 // 2)

        # R -1 -3 -5 -6
        # SPP
        self.conv4 = Conv_Bn_Activation(2048, 512, 1, 1, 'leaky')
        self.conv5 = Conv_Bn_Activation(512, 1024, 3, 1, 'leaky')
        self.conv6 = Conv_Bn_Activation(1024, 512, 1, 1, 'leaky')
        self.conv7 = Conv_Bn_Activation(512, 256, 1, 1, 'leaky')
        # UP
        self.upsample1 = Upsample()
        # R 85
        self.conv8 = Conv_Bn_Activation(512, 256, 1, 1, 'leaky')
        # R -1 -3
        self.conv9 = Conv_Bn_Activation(512, 256, 1, 1, 'leaky')
        self.conv10 = Conv_Bn_Activation(256, 512, 3, 1, 'leaky')
        self.conv11 = Conv_Bn_Activation(512, 256, 1, 1, 'leaky')
        self.conv12 = Conv_Bn_Activation(256, 512, 3, 1, 'leaky')
        self.conv13 = Conv_Bn_Activation(512, 256, 1, 1, 'leaky')
        self.conv14 = Conv_Bn_Activation(256, 128, 1, 1, 'leaky')
        # UP
        self.upsample2 = Upsample()
        # R 54
        self.conv15 = Conv_Bn_Activation(256, 128, 1, 1, 'leaky')
        # R -1 -3
        self.conv16 = Conv_Bn_Activation(256, 128, 1, 1, 'leaky')
        self.conv17 = Conv_Bn_Activation(128, 256, 3, 1, 'leaky')
        self.conv18 = Conv_Bn_Activation(256, 128, 1, 1, 'leaky')
        self.conv19 = Conv_Bn_Activation(128, 256, 3, 1, 'leaky')
        self.conv20 = Conv_Bn_Activation(256, 128, 1, 1, 'leaky')

    def forward(self, input, downsample4, downsample3, inference=False):
        x1 = self.conv1(input)
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        # SPP
        m1 = self.maxpool1(x3)
        m2 = self.maxpool2(x3)
        m3 = self.maxpool3(x3)
        spp = torch.cat([m3, m2, m1, x3], dim=1)
        # SPP end
        x4 = self.conv4(spp)
        x5 = self.conv5(x4)
        x6 = self.conv6(x5)
        x7 = self.conv7(x6)
        # UP
        up = self.upsample1(x7, downsample4.size(), self.inference)
        # R 85
        x8 = self.conv8(downsample4)
        # R -1 -3
        x8 = torch.cat([x8, up], dim=1)

        x9 = self.conv9(x8)
        x10 = self.conv10(x9)
        x11 = self.conv11(x10)
        x12 = self.conv12(x11)
        x13 = self.conv13(x12)
        x14 = self.conv14(x13)

        # UP
        up = self.upsample2(x14, downsample3.size(), self.inference)
        # R 54
        x15 = self.conv15(downsample3)
        # R -1 -3
        x15 = torch.cat([x15, up], dim=1)

        x16 = self.conv16(x15)
        x17 = self.conv17(x16)
        x18 = self.conv18(x17)
        x19 = self.conv19(x18)
        x20 = self.conv20(x19)
        return x20, x13, x6


class Yolov4Head(nn.Module):
    def __init__(self, output_ch, n_classes, inference=False):
        super().__init__()
        self.inference = inference

        self.conv1 = Conv_Bn_Activation(128, 256, 3, 1, 'leaky')
        self.conv2 = Conv_Bn_Activation(256, output_ch, 1, 1, 'linear', bn=False, bias=True)

        self.yolo1 = YoloLayer(
                                anchor_mask=[0, 1, 2], num_classes=n_classes,
                                anchors=[12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401],
                                num_anchors=9, stride=8)

        # R -4
        self.conv3 = Conv_Bn_Activation(128, 256, 3, 2, 'leaky')

        # R -1 -16
        self.conv4 = Conv_Bn_Activation(512, 256, 1, 1, 'leaky')
        self.conv5 = Conv_Bn_Activation(256, 512, 3, 1, 'leaky')
        self.conv6 = Conv_Bn_Activation(512, 256, 1, 1, 'leaky')
        self.conv7 = Conv_Bn_Activation(256, 512, 3, 1, 'leaky')
        self.conv8 = Conv_Bn_Activation(512, 256, 1, 1, 'leaky')
        self.conv9 = Conv_Bn_Activation(256, 512, 3, 1, 'leaky')
        self.conv10 = Conv_Bn_Activation(512, output_ch, 1, 1, 'linear', bn=False, bias=True)

        self.yolo2 = YoloLayer(
                                anchor_mask=[3, 4, 5], num_classes=n_classes,
                                anchors=[12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401],
                                num_anchors=9, stride=16)

        # R -4
        self.conv11 = Conv_Bn_Activation(256, 512, 3, 2, 'leaky')

        # R -1 -37
        self.conv12 = Conv_Bn_Activation(1024, 512, 1, 1, 'leaky')
        self.conv13 = Conv_Bn_Activation(512, 1024, 3, 1, 'leaky')
        self.conv14 = Conv_Bn_Activation(1024, 512, 1, 1, 'leaky')
        self.conv15 = Conv_Bn_Activation(512, 1024, 3, 1, 'leaky')
        self.conv16 = Conv_Bn_Activation(1024, 512, 1, 1, 'leaky')
        self.conv17 = Conv_Bn_Activation(512, 1024, 3, 1, 'leaky')
        self.conv18 = Conv_Bn_Activation(1024, output_ch, 1, 1, 'linear', bn=False, bias=True)

        self.yolo3 = YoloLayer(
                                anchor_mask=[6, 7, 8], num_classes=n_classes,
                                anchors=[12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401],
                                num_anchors=9, stride=32)

    def forward(self, input1, input2, input3):
        x1 = self.conv1(input1)
        x2 = self.conv2(x1)

        x3 = self.conv3(input1)
        # R -1 -16
        x3 = torch.cat([x3, input2], dim=1)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        x6 = self.conv6(x5)
        x7 = self.conv7(x6)
        x8 = self.conv8(x7)
        x9 = self.conv9(x8)
        x10 = self.conv10(x9)

        # R -4
        x11 = self.conv11(x8)
        # R -1 -37
        x11 = torch.cat([x11, input3], dim=1)

        x12 = self.conv12(x11)
        x13 = self.conv13(x12)
        x14 = self.conv14(x13)
        x15 = self.conv15(x14)
        x16 = self.conv16(x15)
        x17 = self.conv17(x16)
        x18 = self.conv18(x17)

        if self.inference:
            y1 = self.yolo1(x2)
            y2 = self.yolo2(x10)
            y3 = self.yolo3(x18)

            return get_region_boxes([y1, y2, y3])

        else:
            return [x2, x10, x18]


class Yolov4(nn.Module):
    def __init__(self, yolov4conv137weight=None, n_classes=80, inference=False):
        super().__init__()

        output_ch = (4 + 1 + n_classes) * 3

        # backbone
        self.down1 = DownSample1()
        self.down2 = DownSample2()
        self.down3 = DownSample3()
        self.down4 = DownSample4()
        self.down5 = DownSample5()
        # neck
        self.neek = Neck(inference)
        # yolov4conv137
        if yolov4conv137weight:
            _model = nn.Sequential(self.down1, self.down2, self.down3, self.down4, self.down5, self.neek)
            pretrained_dict = torch.load(yolov4conv137weight)

            model_dict = _model.state_dict()
            # 1. filter out unnecessary keys
            pretrained_dict = {k1: v for (k, v), k1 in zip(pretrained_dict.items(), model_dict)}
            # 2. overwrite entries in the existing state dict
            model_dict.update(pretrained_dict)
            _model.load_state_dict(model_dict)

        # head
        self.head = Yolov4Head(output_ch, n_classes, inference)


    def forward(self, input):
        d1 = self.down1(input)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)

        x20, x13, x6 = self.neek(d5, d4, d3)

        output = self.head(x20, x13, x6)
        return output


def yolo_forward_dynamic(output, conf_thresh, num_classes, anchors, num_anchors, scale_x_y, only_objectness=1,
                              validation=False):
    # Output would be invalid if it does not satisfy this assert
    # assert (output.size(1) == (5 + num_classes) * num_anchors)

    # print(output.size())

    # Slice the second dimension (channel) of output into:
    # [ 2, 2, 1, num_classes, 2, 2, 1, num_classes, 2, 2, 1, num_classes ]
    # And then into
    # bxy = [ 6 ] bwh = [ 6 ] det_conf = [ 3 ] cls_conf = [ num_classes * 3 ]
    # batch = output.size(0)
    # H = output.size(2)
    # W = output.size(3)

    bxy_list = []
    bwh_list = []
    det_confs_list = []
    cls_confs_list = []

    for i in range(num_anchors):
        begin = i * (5 + num_classes)
        end = (i + 1) * (5 + num_classes)

        bxy_list.append(output[:, begin : begin + 2])
        bwh_list.append(output[:, begin + 2 : begin + 4])
        det_confs_list.append(output[:, begin + 4 : begin + 5])
        cls_confs_list.append(output[:, begin + 5 : end])

    # Shape: [batch, num_anchors * 2, H, W]
    bxy = torch.cat(bxy_list, dim=1)
    # Shape: [batch, num_anchors * 2, H, W]
    bwh = torch.cat(bwh_list, dim=1)

    # Shape: [batch, num_anchors, H, W]
    det_confs = torch.cat(det_confs_list, dim=1)
    # Shape: [batch, num_anchors * H * W]
    det_confs = det_confs.view(output.size(0), num_anchors * output.size(2) * output.size(3))

    # Shape: [batch, num_anchors * num_classes, H, W]
    cls_confs = torch.cat(cls_confs_list, dim=1)
    # Shape: [batch, num_anchors, num_classes, H * W]
    cls_confs = cls_confs.view(output.size(0), num_anchors, num_classes, output.size(2) * output.size(3))
    # Shape: [batch, num_anchors, num_classes, H * W] --> [batch, num_anchors * H * W, num_classes]
    cls_confs = cls_confs.permute(0, 1, 3, 2).reshape(output.size(0), num_anchors * output.size(2) * output.size(3), num_classes)

    # Apply sigmoid(), exp() and softmax() to slices
    #
    bxy = torch.sigmoid(bxy) * scale_x_y - 0.5 * (scale_x_y - 1)
    bwh = torch.exp(bwh)
    det_confs = torch.sigmoid(det_confs)
    cls_confs = torch.sigmoid(cls_confs)

    # Prepare C-x, C-y, P-w, P-h (None of them are torch related)
    grid_x = np.expand_dims(np.expand_dims(np.expand_dims(np.linspace(0, output.size(3) - 1, output.size(3)), axis=0).repeat(output.size(2), 0), axis=0), axis=0)
    grid_y = np.expand_dims(np.expand_dims(np.expand_dims(np.linspace(0, output.size(2) - 1, output.size(2)), axis=1).repeat(output.size(3), 1), axis=0), axis=0)
    # grid_x = torch.linspace(0, W - 1, W).reshape(1, 1, 1, W).repeat(1, 1, H, 1)
    # grid_y = torch.linspace(0, H - 1, H).reshape(1, 1, H, 1).repeat(1, 1, 1, W)

    anchor_w = []
    anchor_h = []
    for i in range(num_anchors):
        anchor_w.append(anchors[i * 2])
        anchor_h.append(anchors[i * 2 + 1])

    device = None
    cuda_check = output.is_cuda
    if cuda_check:
        device = output.get_device()

    bx_list = []
    by_list = []
    bw_list = []
    bh_list = []

    # Apply C-x, C-y, P-w, P-h
    for i in range(num_anchors):
        ii = i * 2
        # Shape: [batch, 1, H, W]
        bx = bxy[:, ii : ii + 1] + torch.tensor(grid_x, device=device, dtype=torch.float32) # grid_x.to(device=device, dtype=torch.float32)
        # Shape: [batch, 1, H, W]
        by = bxy[:, ii + 1 : ii + 2] + torch.tensor(grid_y, device=device, dtype=torch.float32) # grid_y.to(device=device, dtype=torch.float32)
        # Shape: [batch, 1, H, W]
        bw = bwh[:, ii : ii + 1] * anchor_w[i]
        # Shape: [batch, 1, H, W]
        bh = bwh[:, ii + 1 : ii + 2] * anchor_h[i]

        bx_list.append(bx)
        by_list.append(by)
        bw_list.append(bw)
        bh_list.append(bh)


    ########################################
    #   Figure out bboxes from slices     #
    ########################################

    # Shape: [batch, num_anchors, H, W]
    bx = torch.cat(bx_list, dim=1)
    # Shape: [batch, num_anchors, H, W]
    by = torch.cat(by_list, dim=1)
    # Shape: [batch, num_anchors, H, W]
    bw = torch.cat(bw_list, dim=1)
    # Shape: [batch, num_anchors, H, W]
    bh = torch.cat(bh_list, dim=1)

    # Shape: [batch, 2 * num_anchors, H, W]
    bx_bw = torch.cat((bx, bw), dim=1)
    # Shape: [batch, 2 * num_anchors, H, W]
    by_bh = torch.cat((by, bh), dim=1)

    # normalize coordinates to [0, 1]
    bx_bw /= output.size(3)
    by_bh /= output.size(2)

    # Shape: [batch, num_anchors * H * W, 1]
    bx = bx_bw[:, :num_anchors].view(output.size(0), num_anchors * output.size(2) * output.size(3), 1)
    by = by_bh[:, :num_anchors].view(output.size(0), num_anchors * output.size(2) * output.size(3), 1)
    bw = bx_bw[:, num_anchors:].view(output.size(0), num_anchors * output.size(2) * output.size(3), 1)
    bh = by_bh[:, num_anchors:].view(output.size(0), num_anchors * output.size(2) * output.size(3), 1)

    bx1 = bx - bw * 0.5
    by1 = by - bh * 0.5
    bx2 = bx1 + bw
    by2 = by1 + bh

    # Shape: [batch, num_anchors * h * w, 4] -> [batch, num_anchors * h * w, 1, 4]
    boxes = torch.cat((bx1, by1, bx2, by2), dim=2).view(output.size(0), num_anchors * output.size(2) * output.size(3), 1, 4)
    # boxes = boxes.repeat(1, 1, num_classes, 1)

    # boxes:     [batch, num_anchors * H * W, 1, 4]
    # cls_confs: [batch, num_anchors * H * W, num_classes]
    # det_confs: [batch, num_anchors * H * W]

    det_confs = det_confs.view(output.size(0), num_anchors * output.size(2) * output.size(3), 1)
    confs = cls_confs * det_confs

    # boxes: [batch, num_anchors * H * W, 1, 4]
    # confs: [batch, num_anchors * H * W, num_classes]

    return  boxes, confs

class YoloLayer(nn.Module):
    """
    Yolo layer
    model_out: while inference,is post-processing inside or outside the model
        true:outside
    """
    def __init__(self, anchor_mask=[], num_classes=0, anchors=[], num_anchors=1, stride=32, model_out=False):
        super(YoloLayer, self).__init__()
        self.anchor_mask = anchor_mask
        self.num_classes = num_classes
        self.anchors = anchors
        self.num_anchors = num_anchors
        self.anchor_step = len(anchors) // num_anchors
        self.coord_scale = 1
        self.noobject_scale = 1
        self.object_scale = 5
        self.class_scale = 1
        self.thresh = 0.6
        self.stride = stride
        self.seen = 0
        self.scale_x_y = 1

        self.model_out = model_out

    def forward(self, output, target=None):
        if self.training:
            return output
        masked_anchors = []
        for m in self.anchor_mask:
            masked_anchors += self.anchors[m * self.anchor_step:(m + 1) * self.anchor_step]
        masked_anchors = [anchor / self.stride for anchor in masked_anchors]

        return yolo_forward_dynamic(output, self.thresh, self.num_classes, masked_anchors, len(self.anchor_mask),scale_x_y=self.scale_x_y)


def get_region_boxes(boxes_and_confs):

    # print('Getting boxes from boxes and confs ...')

    boxes_list = []
    confs_list = []

    for item in boxes_and_confs:
        boxes_list.append(item[0])
        confs_list.append(item[1])

    # boxes: [batch, num1 + num2 + num3, 1, 4]
    # confs: [batch, num1 + num2 + num3, num_classes]
    boxes = torch.cat(boxes_list, dim=1)
    confs = torch.cat(confs_list, dim=1)

    return boxes, confs

### Download the COCO 2017 Evaluation dataset and define the data loader

In [ ]:
!curl -LO http://images.cocodataset.org/zips/val2017.zip
!curl -LO http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q val2017.zip
!unzip annotations_trainval2017.zip

In [ ]:
import os
import json
import time
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
from pycocotools.coco import COCO


def get_image_filenames(root=os.getcwd()):
    """
    Generate paths to the coco dataset image files.

    Args:
        root (str): The root folder contains.

    Yields:
        filename (str): The path to an image file.
    """
    image_path = os.path.join(root, 'val2017')
    for root, dirs, files in os.walk(image_path):
        for filename in files:
            yield os.path.join(image_path, filename)


def get_coco_dataloader(coco2017_root, transform, subset_indices=None):
    """
    Create the dataset loader and ground truth coco dataset.

    Arguments:
        coco2017_root (str): The root directory to load the data/labels from.
        transform (torchvision.Transform): A transform to apply to the images.
        subset_indices (list): Indices used to create a subset of the dataset.

    Returns:
        loader (iterable): Produces transformed images and labels.
        cocoGt (pycocotools.coco.COCO): Contains the ground truth in coco
            format.
        label_info (dict): A mapping from label id to the human-readable name.
    """

    # Create the dataset
    coco2017_img_path = os.path.join(coco2017_root, 'val2017')
    coco2017_ann_path = os.path.join(
        coco2017_root, 'annotations/instances_val2017.json')

    # check the number of images in val2017 - Should be 5000
    num_files = len(list(get_image_filenames(coco2017_root)))
    print('\nNumber of images in val2017 = {}\n'.format(num_files))

    # load annotations to decode classification results
    with open(coco2017_ann_path) as f:
        annotate_json = json.load(f)
    label_info = {label["id"]: label["name"]
                  for label in annotate_json['categories']}

    # initialize COCO ground truth dataset
    cocoGt = COCO(coco2017_ann_path)

    # create the dataset using torchvision's coco detection dataset
    coco_val_data = dset.CocoDetection(
        root=coco2017_img_path,
        annFile=coco2017_ann_path,
        transform=transform
    )

    if subset_indices is not None:
        # Create a smaller subset of the data for testing - e.g. to pinpoint error at image 516
        coco_val_data = torch.utils.data.Subset(coco_val_data, subset_indices)

    # create the dataloader using torch dataloader
    loader = torch.utils.data.DataLoader(coco_val_data, batch_size=1, shuffle=False)

    return loader, cocoGt, label_info


### Load the COCO dataset

In [ ]:
coco2017_root = './'
orig_coco_val_data_loader, cocoGt, label_info = get_coco_dataloader(coco2017_root, transforms.ToTensor())
transform = transforms.Compose([transforms.Resize([608, 608]), transforms.ToTensor()])
coco_val_data_loader, cocoGt, label_info = get_coco_dataloader(coco2017_root, transform)
image_orig, _ = next(iter(orig_coco_val_data_loader))
print(image_orig.shape)
image, image_info = next(iter(coco_val_data_loader))
image_id = image_info[0]["image_id"].item()
print(image.shape)

### Download pre-trained model checkpoint

In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params={'id': id}, stream=True)
    token = get_confirm_token(response)

    if token:
        params = {'id': id, 'confirm': token}
        response = session.get(URL, params=params, stream=True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
download_file_from_google_drive('1wv_LiFeCRYwtpkqREPeI13-gPELBDwuJ', './yolo_v4.pth')

### Instantiate model and load pretrained checkpoint

In [ ]:
model = Yolov4(yolov4conv137weight=None, n_classes=80, inference=True)
weightfile = "./yolo_v4.pth"
pretrained_dict = torch.load(weightfile, map_location=torch.device('cpu'))
model.load_state_dict(pretrained_dict)
model.eval()

In [ ]:
input_shape = [1,3,608,608]
trace = torch.jit.trace(model.float().eval(), torch.zeros(input_shape).float())
trace.save('model.pth')

In [ ]:
!tar -czvf v2-yolo-model.tar.gz model.pth

### Clean up some files

In [ ]:
!rm annotations_trainval2017.zip
!rm val2017.zip

In [ ]:
!rm model.pth

### Upload the pre-trained checkpoint to Amazon S3

In [ ]:
from sagemaker import get_execution_role
from sagemaker.session import Session

role = get_execution_role()
sess = Session()
region = sess.boto_region_name
bucket = sess.default_bucket()

In [ ]:
sm_client = boto3.client('sagemaker', region_name=region)

In [ ]:
model_filename = 'v2-yolo-model.tar.gz'

In [ ]:
model_path = sess.upload_data(path=model_filename, key_prefix='models')
print(model_path)

### Compile model with SageMaker Neo & package model with SageMaker Edge Manager

In [ ]:
# Specify the path where your model is stored
s3_model_uri = 's3://{}/models/{}'.format(bucket, model_filename)

# Store compiled model in S3 within the 'compiled_models' directory
compilation_output_dir = 'compiled_models'
compiled_output_location = 's3://{}/{}'.format(bucket, compilation_output_dir)

# Store packaged model in S3 within the 'packaged_models' directory
packaged_output_dir = 'packaged_models'
packaged_output_location = 's3://{}/{}'.format(bucket, packaged_output_dir)

In [ ]:
# Framework information
framework = 'PYTORCH'
framework_version = '1.6'

input_shape = [1,3,608,608]
input_layer_name = 'input0'
data_shape = json.dumps({input_layer_name: input_shape})

model_name = 'v2-yolo-model'
model_version = '1.0'

For this demo, we compile the YoloV4 model to run on a C5 EC2 instance (Linux OS, x86_64 arch, Skylake AVX512). A list of supported target architecture types can be found here: https://docs.aws.amazon.com/sagemaker/latest/dg/neo-supported-devices-edge-devices.html

In [ ]:
# Compilation job with unique job ID
from sagemaker.utils import name_from_base
compilation_job_name = name_from_base('torch-yolov4-neo')

sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': s3_model_uri,
        'DataInputConfig': data_shape,
        'Framework': framework,
    },
    OutputConfig={
        'S3OutputLocation': compiled_output_location,
        'TargetPlatform': { 'Os': 'LINUX', 'Arch': 'X86_64' },
        'CompilerOptions': "{ 'mcpu': 'skylake-avx512'}"
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 900 }    
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)

In [ ]:
# Packaging job with unique job ID
edge_packaging_job_name = name_from_base('torch-yolov4-neo')

resp = sm_client.create_edge_packaging_job(
    EdgePackagingJobName=edge_packaging_job_name,
    CompilationJobName=compilation_job_name,
    ModelName=model_name,
    ModelVersion=model_version,
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': packaged_output_location
    }
)
while True:
    resp = sm_client.describe_edge_packaging_job(EdgePackagingJobName=edge_packaging_job_name)    
    if resp['EdgePackagingJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['EdgePackagingJobStatus'], compilation_job_name)        
        break
    time.sleep(5)

## Generate IoT Credentials

For this section, verify that your SageMaker Execution role has the appropriate IoT permissions:
 - iot:CreateThing
 - iot:CreateThingType
 - iot:CreateKeysAndCertificate
 - iot:DescribeRoleAlias
 - iot:CreatePolicy
 - iot:AttachPolicy
 - iot:DescribeEndpoint

In [ ]:
iot_client = boto3.client("iot")

iot_thing_name = "sample-iot-thing"
iot_thing_type = "sample-iot-thing-type"

iot_client.create_thing_type(
    thingTypeName=iot_thing_type
)

# Create an AWS IoT thing objects
iot_client.create_thing(
    thingName=iot_thing_name,
    thingTypeName=iot_thing_type
)

**Note**: Make sure to enter your IoT SageMaker role ARN below!

In [ ]:
device_fleet_name="sample-device-fleet"
fleet_role="<ENTER SAGEMAKER IOT ROLE ARN HERE>" # https://docs.aws.amazon.com/sagemaker/latest/dg/edge-getting-started-step1.html
device_output_bucket=f"s3://{bucket}/device_output/"

In [ ]:
sm_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn=fleet_role,
    Description="fleet description",
    OutputConfig={
        'S3OutputLocation': device_output_bucket
    }
)

In [ ]:
device_name="sample-device"
description="Device 1"

In [ ]:
sm_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {          
            "DeviceName": device_name,
            "IotThingName": iot_thing_name,
            "Description": description
        }
     ]
)

In [ ]:
# Creates a 2048-bit RSA key pair and issues an X.509 # certificate 
# using the issued public key.
create_cert = iot_client.create_keys_and_certificate(
    setAsActive=True 
)

# Get certificate from dictionary object and save in its own
with open('./device.pem.crt', 'w') as f:
    for line in create_cert['certificatePem'].split('\n'):
        f.write(line)
        f.write('\n')
# Get private key from dictionary object and save in its own 
with open('./private.pem.key', 'w') as f:
    for line in create_cert['keyPair']['PrivateKey'].split('\n'):
        f.write(line)
        f.write('\n')
# Get a private key from dictioanary object and save in its own 
with open('./public.pem.key', 'w') as f:
    for line in create_cert['keyPair']['PublicKey'].split('\n'):
        f.write(line)
        f.write('\n')

In [ ]:
# Print Amazon Resource Name (ARN) and alias that has access 
# to AWS Internet of Things (IoT).
sm_client.describe_device_fleet(DeviceFleetName=device_fleet_name)

# Store iot role alias string in a variable
# Grabs role ARN
full_role_alias_name = sm_client.describe_device_fleet(DeviceFleetName=device_fleet_name)['IotRoleAlias']
start_index = full_role_alias_name.find('SageMaker') # Find beginning of role name  
role_alias_name = full_role_alias_name[start_index:]

In [ ]:
role_alias = iot_client.describe_role_alias(
                    roleAlias=role_alias_name)

In [ ]:
alias_policy = {
  "Version": "2012-10-17",
  "Statement": {
    "Effect": "Allow",
    "Action": "iot:AssumeRoleWithCertificate",
    "Resource": role_alias['roleAliasDescription']['roleAliasArn']
  }
}

policy_name = 'aliaspolicy-'+ str(time.time()).split('.')[0]
aliaspolicy = iot_client.create_policy(policyName=policy_name,
                                       policyDocument=json.dumps(alias_policy))

# Attach policy
iot_client.attach_policy(policyName=policy_name,
                            target=create_cert['certificateArn'])

In [ ]:
# Get the unique endpoint specific to your AWS account that is making the call.
iot_endpoint = iot_client.describe_endpoint(
    endpointType='iot:CredentialProvider'
)

endpoint="https://{}/role-aliases/{}/credentials".format(iot_endpoint['endpointAddress'],role_alias_name)

In [ ]:
!wget https://www.amazontrust.com/repository/AmazonRootCA1.pem

In [ ]:
!curl --cert device.pem.crt --key private.pem.key --cacert AmazonRootCA1.pem $endpoint

In [ ]:
!aws s3 cp private.pem.key s3://{bucket}/authorization-files/
!aws s3 cp device.pem.crt s3://{bucket}/authorization-files/
!aws s3 cp AmazonRootCA1.pem s3://{bucket}/authorization-files/

### Download & configure SageMaker Edge Agent

In [ ]:
agent_version="1.20210512.96da6cc"
agent_arch="linux-x64"

In [ ]:
!aws s3 cp s3://sagemaker-edge-release-store-{region}-{agent_arch}/Releases/{agent_version}/{agent_version}.tgz ./

In [ ]:
!aws s3 cp s3://sagemaker-edge-release-store-{region}-{agent_arch}/Certificates/{region}/{region}.pem ./

In [ ]:
!aws s3 cp {region}.pem s3://{bucket}/authorization-files/

In [ ]:
sagemaker_edge_config = {
    "sagemaker_edge_core_device_name": device_name,
    "sagemaker_edge_core_device_fleet_name": device_fleet_name,
    "sagemaker_edge_core_capture_data_buffer_size": 30,
    "sagemaker_edge_core_capture_data_push_period_seconds": 4,
    "sagemaker_edge_core_folder_prefix": "demo_capture",
    "sagemaker_edge_core_region": region,
    "sagemaker_edge_core_root_certs_path": "/home/agent/certs/root",
    "sagemaker_edge_provider_aws_ca_cert_file": "/home/agent/certs/iot/AmazonRootCA1.pem",
    "sagemaker_edge_provider_aws_cert_file": "/home/agent/certs/iot/device.pem.crt",
    "sagemaker_edge_provider_aws_cert_pk_file": "/home/agent/certs/iot/private.pem.key",
    "sagemaker_edge_provider_aws_iot_cred_endpoint": endpoint,
    "sagemaker_edge_provider_provider": "Aws",
    "sagemaker_edge_provider_s3_bucket_name": bucket,
    "sagemaker_edge_core_capture_data_destination": "Cloud"
}

In [ ]:
edge_config_file = open("agent.json", "w")
json.dump(sagemaker_edge_config, edge_config_file, indent = 6)
edge_config_file.close()

### SageMaker Edge Agent Deployment Package set-up

Now that we have the required files for the edge agent to interact with the cloud, we can create the correct directory structure and zip up the files for use when building our Docker container. The directory structure should match the following:

In [ ]:
!mkdir agent
!mkdir agent/app
!mkdir agent/artifacts
!mkdir agent/certs
!mkdir agent/certs/root
!mkdir agent/certs/iot
!mkdir agent/conf
!mkdir agent/models
!mkdir agent/models/{device_name}
!mkdir agent/models/{device_name}/{model_name}
!mkdir agent/models/{device_name}/{model_name}/{model_version}

!mv AmazonRootCA1.pem agent/certs/iot/
!mv device.pem.crt agent/certs/iot/
!mv private.pem.key agent/certs/iot/

!mv {region}.pem agent/certs/root/

!mv agent.json agent/conf/

!mv {agent_version}.tgz agent/
!tar -zxf agent/{agent_version}.tgz -C agent/artifacts/

### Build client API stubs

Let's extract the edge agent from the tgz file and generate the client API stubs that our driver application will use to interact with the model via gRPC or Shared Memory. For more details on the API definition, please visit the documentation here: https://docs.aws.amazon.com/sagemaker/latest/dg/edge-manage-model.html

In [ ]:
# by using protoc, we can generate stubs (client api) for connecting to the agent and invoking its API
!python3 -m grpc_tools.protoc --proto_path=agent/artifacts/docs/api --python_out=agent/app/ --grpc_python_out=agent/app/ agent/artifacts/docs/api/agent.proto

We can remove the artifacts directory for now since our Docker build process will take care of downloading the agent artifacts for us.

In [ ]:
!rm -rf agent/artifacts

Now let's zip up this main part of the deployment package and upload it to S3. Then, we are ready to download the artifacts and build our agent container. You can either do this locally or as part of your container pipeline.

In [ ]:
deployment_package_name='agent_deployment_package.tar.gz'

In [ ]:
!tar -czvf {deployment_package_name} agent --recursive

In [ ]:
agent_deployment_path = sess.upload_data(path=deployment_package_name, key_prefix='agent_deployment')
print(agent_deployment_path)

In [ ]:
!aws s3 mv s3://{bucket}/packaged_models/{model_name}-{model_version}.tar.gz s3://{bucket}/agent_deployment/{model_name}-{model_version}.tar.gz

### Complete the remaining steps locally or in your container build pipeline
The remaining steps require Docker to build the container images for the edge agent and the driver application. Follow the repo README for instructions for the remaining steps.